In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

# Define the CNN architecture
class Deep_Emotion(nn.Module):
    def __init__(self):
        super(Deep_Emotion, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.conv2 = nn.Conv2d(10, 10, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(10, 10, 3)
        self.conv4 = nn.Conv2d(10, 10, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810, 50)
        self.fc2 = nn.Linear(50, 7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self, input):
        out = self.stn(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [9]:
transform = transforms.Compose([
    transforms.Resize((48, 48)),  
    transforms.Grayscale(),       
    transforms.ToTensor(),        
    transforms.Normalize((0.5,), (0.5,)) 
])
train_data_path = 'face/train'
test_data_path = 'face/test'
train_dataset = ImageFolder(train_data_path, transform=transform)
test_dataset = ImageFolder(test_data_path, transform=transform)
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Deep_Emotion().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

best_loss = float('inf')
patience = 5
counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        tepoch.set_description(f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            tepoch.set_postfix(loss=running_loss / total, accuracy=correct / total)
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
        torch.save(model.state_dict(), 'emotion_model.pth')
    else:
        counter += 1
    if counter >= patience:
        print(f'Early stopping at epoch {epoch+1} as validation loss did not improve for {patience} consecutive epochs.')
        break

Epoch 1/100: 100%|█| 225/225 [08:29<00:00,  2.27s/batch, accuracy=0.311, loss=1.


Epoch 1/100, Loss: 1.7132, Accuracy: 0.3108


Epoch 2/100: 100%|█| 225/225 [07:03<00:00,  1.88s/batch, accuracy=0.392, loss=1.


Epoch 2/100, Loss: 1.5615, Accuracy: 0.3923


Epoch 3/100: 100%|█| 225/225 [06:54<00:00,  1.84s/batch, accuracy=0.431, loss=1.


...

Epoch 65/100, Loss: 1.0706, Accuracy: 0.5908


Epoch 66/100: 100%|█| 225/225 [06:40<00:00,  1.78s/batch, accuracy=0.596, loss=1


Epoch 66/100, Loss: 1.0598, Accuracy: 0.5963


Epoch 67/100: 100%|█| 225/225 [06:49<00:00,  1.82s/batch, accuracy=0.595, loss=1

Epoch 67/100, Loss: 1.0616, Accuracy: 0.5946
Early stopping at epoch 67 as validation loss did not improve for 5 consecutive epochs.


In [1]:
import cv2
import torch
import numpy as np
from torchvision.transforms import transforms
from PIL import Image

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

# Function to detect and recognize emotions from an image
def detect_emotion(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # For each detected face, crop, resize, and recognize emotion
    for (x, y, w, h) in faces:
        # Crop the face region
        face = gray[y:y+h, x:x+w]
        # Resize the face image to match the input size of your CNN model
        face = cv2.resize(face, (48, 48))

        face_pil = Image.fromarray(face)
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        face_tensor = transform(face_pil).unsqueeze(0)
        model.eval()
        with torch.no_grad():
            outputs = model(face_tensor)
            _, predicted = torch.max(outputs, 1)

        emotion_label = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}[predicted.item()]
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(image, emotion_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Emotion Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Call the function with the image path
detect_emotion('images/IMG_0125.jpg')


/home/aldrax/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


NameError: name 'model' is not defined

In [ ]:
import cv2
import torch
from torchvision.transforms import transforms
from PIL import Image
# outputs = model(face_tensor)
image_path = 'images/IMG_0125.jpg'
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

In [4]:
for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face_pil = Image.fromarray(face)
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        face_tensor = transform(face_pil).unsqueeze(0)
face_tensor

NameError: name 'faces' is not defined

In [14]:
model.eval()
with torch.no_grad():
    outputs = model(face_tensor)
    _, predicted = torch.max(outputs, 1)

/home/aldrax/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/aldrax/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [17]:
print(outputs)
predicted

tensor([[ 0.7549, -3.5763,  0.3546, -0.2624,  1.1595,  1.0852, -0.5098]])


tensor([4])